In [19]:
import requests
import csv
from bs4 import BeautifulSoup

# Read stock symbols from stocks.txt
with open("stocks.txt", "r") as file:
    stock_symbols = [line.strip() for line in file.readlines()]

# Define headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0"
}

# Excluded categories
exclude_keywords = ["Life", "Entertainment", "Finance", "Sports", "New on Yahoo"]

# Prepare list for storing news
all_news = []

for ticker in stock_symbols:
    url = f"https://finance.yahoo.com/quote/{ticker}/news/"
    print(f"🔍 Fetching news for {ticker} from {url}")

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find all news headlines
        headlines = soup.find_all("h3")

        for h in headlines:
            headline_text = h.get_text(strip=True)

            # Filter out unwanted categories
            if not any(keyword in headline_text for keyword in exclude_keywords):
                all_news.append([ticker, headline_text])

    else:
        print(f" Failed to fetch  {ticker}. Status code:", response.status_code)

# Save to CSV
filename = "news.csv"
with open(filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Ticker", "Headline"])  # Write header
    writer.writerows(all_news)

print(f" All stock news saved in {filename}")


🔍 Fetching news for AAPL from https://finance.yahoo.com/quote/AAPL/news/
🔍 Fetching news for MSFT from https://finance.yahoo.com/quote/MSFT/news/
🔍 Fetching news for GOOGL from https://finance.yahoo.com/quote/GOOGL/news/
🔍 Fetching news for ^GSPC from https://finance.yahoo.com/quote/^GSPC/news/
🔍 Fetching news for NVDA from https://finance.yahoo.com/quote/NVDA/news/
🔍 Fetching news for TSLA from https://finance.yahoo.com/quote/TSLA/news/
🔍 Fetching news for PLTR from https://finance.yahoo.com/quote/PLTR/news/
🔍 Fetching news for LCID from https://finance.yahoo.com/quote/LCID/news/
🔍 Fetching news for AAL from https://finance.yahoo.com/quote/AAL/news/
🔍 Fetching news for SOFI from https://finance.yahoo.com/quote/SOFI/news/
 All stock news saved in news.csv


In [20]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Step 1: Read the scraped news from news.csv
df = pd.read_csv("news.csv")

# Step 2: Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Step 3: Process sentiment analysis
sentiment_results = []

for _, row in df.iterrows():
    ticker = row["Ticker"]
    headline = str(row["Headline"]).strip()

    if headline.lower() == "nan" or headline == "":  
        continue  

    sentiment_score = analyzer.polarity_scores(headline)["compound"]
    sentiment_label = (
        "Positive" if sentiment_score > 0 else 
        "Negative" if sentiment_score < 0 else 
        "Neutral"
    )
    
    sentiment_results.append([ticker, sentiment_score, sentiment_label])

# Step 4: Save results to sentiment.csv
sentiment_df = pd.DataFrame(sentiment_results, columns=["Ticker/Headline", "Sentiment Score", "Sentiment"])
sentiment_df.to_csv("sentiment.csv", index=False)

print(" Sentiment analysis saved in 'sentiment.csv' in a clean table format!")


 Sentiment analysis saved in 'sentiment.csv' in a clean table format!
